In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import prince

In [11]:
df = pd.read_csv('matriculados_limpia.csv')
##mostrar todas las columnas con un registro
pd.set_option('display.max_columns', None)
df.head(1)
##mostrar las variables que toma cada columna, todos sus posibles valores
for col in df.columns:
    print(col, df[col].unique())

YEAR [2019 2023 2020 2021 2022]
SEMESTRE [1 2]
TIPO_NIVEL ['Pregrado' 'Postgrado']
NIVEL ['Pregrado' 'Maestría' 'Especialidades médicas' 'Especialización'
 'Doctorado']
DEP_NAC ['BOGOTÁ, D. C.' 'CUNDINAMARCA' 'META' 'Sin información' 'CALDAS'
 'GUAINÍA' 'NARIÑO' 'BOYACÁ' 'TOLIMA' 'HUILA' 'SANTANDER'
 'NORTE DE SANTANDER' 'PUTUMAYO' 'BOLÍVAR' 'VALLE DEL CAUCA' 'ARAUCA'
 'GUAVIARE' 'CASANARE' 'QUINDÍO' 'ANTIOQUIA' 'VICHADA' 'CAQUETÁ'
 'ATLÁNTICO' 'CESAR' 'AMAZONAS' 'CAUCA'
 'ARCHIPIÉLAGO DE SAN ANDRÉS, PROVIDENCIA Y SANTA CATALINA' 'MAGDALENA'
 'LA GUAJIRA' 'RISARALDA' 'CÓRDOBA' 'SUCRE' 'VAUPÉS' 'CHOCÓ']
CIU_NAC ['BOGOTÁ, D.C.' 'FACATATIVÁ' 'VILLAVICENCIO' 'Sin información' 'MANIZALES'
 'INÍRIDA' 'IPIALES' 'DUITAMA' 'IBAGUÉ' 'NEIVA' 'CHÍA' 'UBALÁ'
 'MANZANARES' 'TUNJA' 'ARMERO GUAYABAL' 'GIRARDOT' 'ALDANA' 'BUCARAMANGA'
 'CÚCUTA' 'COLÓN' 'PITALITO' 'SOGAMOSO' 'VILLA DE SAN DIEGO DE UBATÉ'
 'ACACÍAS' 'GACHETÁ' 'SANTA ROSA DEL SUR' 'SUCRE' 'SOACHA' 'CALI' 'PASTO'
 'PALMIRA' 'CHOACHÍ' 'COLO

In [7]:
df.dtypes


YEAR                  int64
SEMESTRE              int64
TIPO_NIVEL           object
NIVEL                object
DEP_NAC              object
CIU_NAC              object
LON_CIU_NAC         float64
LAT_CIU_NAC         float64
DEP_PROC             object
CIU_PROC             object
LON_CIU_PROC        float64
LAT_CIU_PROC        float64
CODS_NAC             object
NACIONALIDAD         object
EDAD                  int64
SEXO                 object
ESTRATO              object
TIPO_COL             object
PBM                   int64
MAT_PVEZ             object
SEDE_NOMBRE_ADM      object
SEDE_NOMBRE_MAT      object
ADM_PEAMA_ANDINA     object
TIPO_ADM             object
PAES                 object
PEAMA                object
MOV_PEAMA            object
CONVENIO             object
TIP_CONVENIO         object
FACULTAD             object
SNIES_PROGRA          int64
PROGRAMA             object
AREAC_SNIES          object
CA_CINE               int64
CD_CINE               int64
AREA_CINE           

# Convertir variables ordinales
La columna ESTRATO está almacenada como tipo object (texto), lo que implica que sus valores son cadenas 

Los algoritmos de clustering trabajan con números, por lo que es necesario convertirla a un formato numérico, como es ordinal, se puede hacer un map.

In [4]:

# Convertir variables ordinales
#ver que valores toma la columna de estrato
df["ESTRATO"].unique()
#mapear los valores de estrato a numeros
estrato_mapping = {"Estrato 1": 1, "Estrato 2": 2, "Estrato 3": 3, "Estrato 4": 4, "Estrato 5": 5, "Estrato 6": 6, "ND/NE": 0}
df["ESTRATO"] = df["ESTRATO"].map(estrato_mapping)

array(['Estrato 3', 'Estrato 4', 'Estrato 2', 'Estrato 1', 'ND/NE',
       'Estrato 5', 'Estrato 6'], dtype=object)

### Clasificación de las columnas

---

#### **1. `numeric_cols` (Variables Numéricas)**
- **Columnas incluidas**: `EDAD`, `PBM`, `LON_CIU_NAC`, `LAT_CIU_NAC`.
- **Razón**:
  - Son variables **cuantitativas** (números continuos o discretos).
  - Requieren **escalado** (ej: estandarización) para que todas tengan la misma importancia en el clustering.
  - Ejemplo: `EDAD` y `PBM` son valores numéricos puros, mientras que `LON_CIU_NAC` y `LAT_CIU_NAC` son coordenadas geográficas.

---

#### **2. `categorical_low` (Variables Categóricas de Baja Cardinalidad)**
- **Columnas incluidas**: `SEXO`, `TIPO_ADM`, `PAES`.
- **Definición de "baja cardinalidad"**:
  - Tienen **pocas categorías únicas** (ej: `SEXO` con valores como "Masculino" y "Femenino").
  - Generalmente menos de 10 categorías.
- **Razón para separarlas**:
  - **One-Hot Encoding no genera alta dimensionalidad**: Convertirlas en variables dummy (0/1) no añade demasiadas columnas al dataset.
  - Ejemplo: Si `SEXO` tiene 2 categorías, generará solo 2 columnas binarias.
  - Son fáciles de manejar sin técnicas especiales.

---

#### **3. `categorical_high` (Variables Categóricas de Alta Cardinalidad)**
- **Columnas incluidas**: `DEP_NAC`, `CIU_NAC`, `FACULTAD`, `PROGRAMA`.
- **Definición de "alta cardinalidad"**:
  - Tienen **muchas categorías únicas** (ej: `CIU_NAC` con cientos de ciudades o `PROGRAMA` con decenas de programas académicos).
  - Generalmente más de 10 categorías.
- **Razón para separarlas**:
  - **One-Hot Encoding puede ser problemático**: Si una columna tiene 100 categorías, generará 100 nuevas columnas, lo que aumenta la dimensionalidad y afecta el rendimiento del modelo.
  - Requieren **técnicas alternativas** para evitar la "maldición de la dimensionalidad":
    - **Frequency Encoding**: Reemplazar categorías por su frecuencia.
    - **Target Encoding**: Usar la media de una variable numérica relacionada.
    - **Embeddings**: Representaciones densas aprendidas (ej: Word2Vec para nombres de facultades).

---


In [12]:
numeric_cols = ["YEAR", "SEMESTRE", "LON_CIU_NAC", "LAT_CIU_NAC", 
                "LON_CIU_PROC", "LAT_CIU_PROC", "EDAD", "PBM", "ESTRATO", "CA_CINE"]

In [ ]:
categorical_low = ["TIPO_NIVEL", "NIVEL", "NACIONALIDAD", "SEXO",
    "TIPO_COL", "MAT_PVEZ", "SEDE_NOMBRE_ADM", "SEDE_NOMBRE_MAT",
    "ADM_PEAMA_ANDINA", "TIPO_ADM", "PAES", "PEAMA",
    "MOV_PEAMA", "CONVENIO", "TIP_CONVENIO", "AREAC_SNIES", "AREA_CINE"]

for col in categorical_low:
    print(col, df[col].nunique())


TIPO_NIVEL 2
NIVEL 5
NACIONALIDAD 2
SEXO 2
TIPO_COL 4
MAT_PVEZ 2
SEDE_NOMBRE_ADM 9
SEDE_NOMBRE_MAT 9
ADM_PEAMA_ANDINA 5
TIPO_ADM 3
PAES 7
PEAMA 8
MOV_PEAMA 3
CONVENIO 3
TIP_CONVENIO 3
AREAC_SNIES 8
AREA_CINE 9


In [20]:
categorical_high = ["DEP_NAC", "CIU_NAC", "DEP_PROC", "CIU_PROC",
    "FACULTAD", "PROGRAMA", "CD_CINE"]

for col in categorical_high:
    print(col, df[col].nunique())

DEP_NAC 34
CIU_NAC 951
DEP_PROC 34
CIU_PROC 900
FACULTAD 22
PROGRAMA 521
CD_CINE 57


In [ ]:



# 4. Crear un pipeline de preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols),
        ("cat_low", OneHotEncoder(handle_unknown="ignore"), categorical_low),
        ("cat_high", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_high),
    ],
    remainder="passthrough"
)

# 5. Reducción de dimensionalidad con FAMD (maneja datos mixtos)
pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("famd", prince.FAMD(n_components=10, random_state=42)),
    ]
)

X_transformed = pipeline.fit_transform(df)